<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


In [0]:
# Zahlungsauftrag Generator

import numpy as np
import pandas as pd


# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv('https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv', delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample']

# IBAN Gen. Erster Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))

iban_country_code = code + checkdig

iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)

iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. Zweiter Teil - Erstellung Kontonummer
eiban = []

for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_country_code + iban_bank_code + part_eiban.astype(str)
    

eiban = pd.DataFrame(eiban)#,index=None)

eiban.drop([0])
eiban = np.ravel(eiban)

eiban = np.random.choice(eiban)
eiban
#eiban.to_csv('gen_iban.csv',index=False)




#data = pd.read_csv('gen_iban.csv',header=None, index_col=False)


In [0]:
# IBAN Generator

import numpy as np
import pandas as pd


iban_country_code = 'CH53'
iban_bank_code = '0077'

 
iban_cc = pd.read_csv('https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv', delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample']

#cc_ch = iban_cc['Code'].where(iban_cc['Code'])

iban_account1 = np.int64(np.random.randint(2000,3000,20800))
iban_account2 = np.int64(np.random.randint(3000,4000,20800))
iban_account3 = np.int64(np.random.randint(5000,6000,20800))
iban_account4 = np.random.randint(1,9,20800)


for row in range(len(eiban)):
  if row > 1:
    iban_country_code + iban_bank_code + eiban.astype(str))


iban = {'a':iban_account1,'b':iban_account2 ,'c':iban_account3 ,'d':iban_account4}
iban = pd.DataFrame(iban)
ccode = []
bcode = []
eiban = []

for row in range(len(iban)):
  if row > 1:
    ccode.append(iban_country_code)
    bcode.append(iban_bank_code)
   
cc = pd.DataFrame(ccode)
bc = pd.DataFrame(bcode)
iban.insert(0,'cc',cc)
iban.insert(1,'bc',bc)



#iban.to_csv('gen_iban.csv',index=False, sep=' ')

#data = pd.read_csv('gen_iban.csv',header=None, index_col=False)





In [0]:
# IBAN Generator in elektronischer Form

import numpy as np
import pandas as pd



iban_cc = pd.read_csv('https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv', delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample']

iban_code = pd.DataFrame(iban_cc['Code'], columns=['Code'])
np.ravel(iban_cc['Code'])

#code = np.random.choice(np.ravel(iban_code['Code']).astype(str))
#checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))

#iban_bank_code = iban_cc['Length'].where(iban_cc['Code'] = code)


#iban_bank_code = np.int64(np.random.randint(11, 99,1))

iban_country_code = code + checkdig



eiban = []

for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_country_code + iban_bank_code + part_eiban.astype(str)
    

eiban = pd.DataFrame(eiban)#,index=None)

eiban.drop([0])
eiban = np.ravel(eiban)

eiban = np.random.choice(eiban)
eiban
#eiban.to_csv('gen_iban.csv',index=False)




#data = pd.read_csv('gen_iban.csv',header=None, index_col=False)


ValueError: ignored